# Inverse Ising Problem
This code infers the interaction parameters of the classical hamiltonian from the statistics using a classical Boltzmann machine.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Since the free statistics have to be calculated at each iteration of the gradient ascent loop, we want our functions to calculate the free statistics as efficient as possible. We first start by simplifying the solver functions defined for the forward Ising problem. We can recycle a lot of the code that we used for the forward Ising problem.

The inverse problem is to learn the values of the weight matrix using the statistics of the system. This was done by using a Boltzmann machine, which physically corresponds to an 2D Ising model at finite temperature.

In [2]:
# WEIGHT FUNCTIONS
#-----------------------------------------------------------------------------------------------------------
def generate_weights_nearest_neighbour(L, w, h):
    '''Generates weights array with only nearest neighbour interactions and periodic boundary conditions''' 
    N = L**2
    W = np.zeros((N, N))
    for i in range(L):
        for j in range(L):
            site  = L * i             + j
            east  = L * i             + (j + 1) % L
            west  = L * i             + (j - 1) % L
            north = L * ((i + 1) % L) + j
            south = L * ((i - 1) % L) + j
            
            W[site, east]  += w
            W[site, west]  += w
            W[site, north] += w
            W[site, south] += w
            W[site, site]  += h
    return W

def generate_weights_fully_connected(L, w, h):
    '''Generates weights array for a fully connected system'''
    N = L**2
    W = np.ones((N, N)) * w 
    np.fill_diagonal(W, h)
    return W / N

def generate_weights_random(L, w, h, seed):
    '''Generates a symmetric random weights array for a given seed'''
    N = L**2      
    rng = np.random.default_rng(seed)   #  set random seed for reproduciblity
    W_assym = rng.random((N, N)) * w    #  get random weight matrix
    W = (W_assym+ W_assym.T)/2          #  symmetrize the random matrix
    return W 

In [3]:
###  SIMULATION HELPER FUNCTIONS
#----------------------------------------------------------------------
def initial_config(L):   
    '''Generates a 1D array of length N with random binary spin values'''
    return np.random.choice([-1, 1], size=(L**2))

def all_possible_configs(L):
    '''Generates all possible configurations of a binary spin system of size N'''
    N = L**2
    configs = np.zeros((2**N,N))
    for i in range(2**N):
        config = np.array([1 if x == '1' else -1 for x in np.binary_repr(i, width=N)])
        configs[i] = config
    return configs

def calcEnergy(config, W):
    '''Calculates the energy of a given configuration'''
    energy = -np.dot(config.T,np.dot(W, config)) / 2 - np.dot(np.diagonal(W),config)
    return energy

def calcCorr(config): 
    '''Calculates the spin-spin correlations of a given configuration. Returns a NxN matrix with the correlations'''
    return np.outer(config, config.T)

def heat_bath_move(config, W, beta):
    '''Implements a Monte Carlo sweep using the heat bath algorithm'''
    N = config.shape[0]
    for i in range(N):                                 #  this for loop ensures we perform a sweep
        a  = np.random.randint(N)                      #  choose a random spin i 
        lf = np.dot(config[a], np.dot(W[a],config))    #  calculate local field energy of spin i
        p  = np.exp(-2 * beta * lf)                    #  probability of flipping spin i
        if np.random.rand() < p:
            config[a] = -config[a]
    return config

In [4]:
###  GET SIMPLIFIED SOLVER FUNCTIONS
#----------------------------------------------------------------------
def ising_solve_exact_simplified(L, W, T, configs):    
    '''Calculates the free statistics for a single temperature using exact calculation'''
    N = L**2
    beta = 1.0/T 
    Z, C1, C2  = 0, np.zeros((N)), np.zeros((N,N))        #  initialize arrays to store variables                       
                                           
    for config in configs:
        Ene  = calcEnergy(config,W)
        Corr = calcCorr(config) 
        p    = np.exp(-beta * Ene)                        #  non-normalized probability of this 
                                                          #  configuration at this temperature
        Z  +=  p                                        
        C1 +=  p * config                               
        C2 +=  p * Corr

    Ci   = C1 / Z
    CiCj = C2 / Z
        
    return Ci,CiCj

def ising_solve_mc_simplified(L, W, T, config, mcSteps):    
    '''Calculate the free statistics for a single temperature using Monte Carlo Heat Bath sampling'''
    N = L**2
    beta = 1.0/T 
    Ci,CiCj = 0,np.zeros((N)),np.zeros((N,N))             #  initialize arrays to store variables 

    for i in range(mcSteps):   
        config      = heat_bath_move(config, W, beta)     #  performs a sweep. no need to equilibrate.               
        Corr        = calcCorr(config)             

        C1   += config
        C2   += Corr
   
    Ci[tt]    = C1 / mcSteps
    CiCj[tt]  = C2 / mcSteps
        
    return Ci,CiCj

I've checked and these functions recover the exact same values for Ci and CiCj as the more expansive version does.

In [68]:
## SOLVE USING EXACT COMPUTATION 
#----------------------------------------------------------------------
def solve_inverse_ising_exact(L, T, w, h, eqSteps, mcSteps, W_type, Clamp_type, Free_type, threshold, maxiter, eta):
    '''Solves the inverse Ising problem. Returns weight matrices that generated the clamped statistics 
       and the inferred weight matrix'''
    
    #  initialize weight matrix
    if W_type == 0:                                           
        W_clamp = generate_weights_random(L,w,h,777)           #  random seed 777
    elif W_type == 1:                                         
        W_clamp = generate_weights_nearest_neighbour(L,w,h)
    elif W_type == 2:
        W_clamp = generate_weights_fully_connected(L,w,h) 
    else:
        print("Non-valid weight matrix type")
        
    #  generate the clamped statistics based on solver type
    if Free_type == 0:
        configs = all_possible_configs(L)
        Ci_clamp, CiCj_clamp = ising_solve_exact_simplified(L, W_clamp, T, configs)
    if Free_type == 1:
        config = initial_config(L)                             #  intialize a state
        for i in range(eqSteps):                               #  equilibrate
            config    = heat_bath_move(config, W, beta)
            Ci_clamp, CiCj_clamp = ising_solve_exact_simplified(L, W_clamp, T, config, eqSteps)
            
            #  initialize a new state to generate free statistics from later on
            config = initial_config(L)                            
            for i in range(eqSteps):                           #  we only want to equiliberate once
                config    = heat_bath_move(config, W, beta)    #  to be more efficient    

    #  get random initial weights that generate free statistics 
    W = generate_weights_random(L, w, h, 333)                  #  random seed 333
    H = W.diagonal()                                           #  get self-interactions
    
    #  initialize gradient ascent 
    N    = L**2
    beta = 1.0 / T 
    Ci_free, CiCj_free = np.zeros((N)), np.zeros((N,N))        
    it = 0
    d_W = d_H = 1 
    
    while d_W + d_H > threshold and it < maxiter: #  start gradient ascent
        it += 1

        #generate free statistics based on type of solver
        if Free_type == 0:
            Ci_free,CiCj_free = ising_solve_exact_simplified(L, W, T, configs) 
        elif Free_type == 1:
            Ci_free,CiCj_free = ising_solve_mc_simplified(L, W, T, config, mcSteps)
            
        H     = H + eta * (Ci_clamp   - Ci_free)               #  update weight matrix based on statistics  
        W     = W + eta * (CiCj_clamp - CiCj_free)

        d_H = np.abs(np.sum(Ci_free - Ci_clamp))
        d_W = np.abs(np.sum(CiCj_free - CiCj_clamp))

        np.fill_diagonal(W, H)                                 #  update diagonal weights to W

    loopdata = [it, d_H, d_W]
    return W, W_clamp, loopdata

In [79]:
##  PARAMETERS
#----------------------------------------------------------------------

# model parameters
L           = 2                            #  linear dimension of the LxL lattice
T           = 1                            #  temperature scalar
w           = 1                            #  strength of interactions 
h           = 0                            #  strength of self-interactions
eqSteps     = 2**9                         #  number of MC sweeps for equilibration
mcSteps     = 2**10                        #  number of MC sweeps for calculation
W_type      = 0                            #  0 for random weights, 1 for nb, 2 for fully connected
Clamp_type  = 0                            #  determine how the clamped statistics are generated (0 exact, 1 mc)
Free_type   = 0                            #  determine how the free statistics are generated (0 exact, 1 mc)

# gradient ascent parameters 
threshold    = 1e-8     #  threshold for stopping the gradient ascent loop
maxiter      = 2**11    #  maximum number of gradient ascent iterations
eta          = 1/2      #  learning rate

In [80]:
W, W_clamp, loopdata = solve_inverse_ising_exact(3, T, w, h, eqSteps, mcSteps, 
                                                 W_type, Clamp_type, Free_type, threshold, maxiter, eta)

In [81]:
print(np.abs(W - W_clamp))
print(loopdata)
np.set_printoptions(formatter={'float_kind':"{:.2f}".format}) # print matrices in 2 decimals

[[0.08 0.19 0.22 0.08 0.25 0.06 0.07 0.09 0.16]
 [0.19 0.04 0.35 0.41 0.09 0.16 0.09 0.31 0.12]
 [0.22 0.35 0.34 0.21 0.12 0.00 0.15 0.29 0.05]
 [0.08 0.41 0.21 0.01 0.21 0.13 0.28 0.50 0.54]
 [0.25 0.09 0.12 0.21 0.13 0.39 0.06 0.17 0.38]
 [0.06 0.16 0.00 0.13 0.39 0.06 0.09 0.11 0.13]
 [0.07 0.09 0.15 0.28 0.06 0.09 0.32 0.10 0.04]
 [0.09 0.31 0.29 0.50 0.17 0.11 0.10 0.39 0.01]
 [0.16 0.12 0.05 0.54 0.38 0.13 0.04 0.01 0.35]]
[2048, 5.189969323193822e-07, 9.5084934405687e-05]


### Results

Exact computation performs well for N = 2. Noticable dropoff when going to N = 3. It finds completely different configurations that generate approximately the same statistics.

The algorithm performs best for fully connected random weights. It has some problems with 0 values. It can find the nearest neighbour matrix but only after quite a long time (750.000 iterations)

When increasing the lattice size, the computation time ....

For random weights it appears that a high learning rate (~1 seems optimal) but this can be investigated further.

At first, sove the inverse problem for 1 temperature at the time. Then scale up to multiple. Seperate the calculation of intensive values and correlations. You only need the correlations here.